In [8]:
import os
import cv2
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from ultralytics import YOLO
from scipy.signal import savgol_filter


def smooth_data(data, window_size=5, polyorder=2):
    if len(data) < window_size:
        return data  # No suficiente data para suavizar
    return savgol_filter(data, window_size, polyorder, axis=0)

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def calculate_distance(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def extract_pose_features(video_path, model, scaler_X, encoder, modelYolo):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        alto_original, ancho_original = frame.shape[:2]
        nuevo_alto = 1100  # Nueva altura deseada
        nuevo_ancho = int((nuevo_alto / alto_original) * ancho_original)
        frame = cv2.resize(frame, (nuevo_ancho, nuevo_alto))

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = modelYolo(frame)

        keypoints = results[0].keypoints.data.numpy().tolist()[0]
        
        # Definir las conexiones entre keypoints
        connections = [(5, 6), (5, 7), (6, 8), (7, 9), (8, 10), (11, 12), (5, 11), (6, 12), (11, 13), (12, 14), (13, 15), (14, 16)]

        for connection in connections:
            start_keypoint = connection[0]
            end_keypoint = connection[1]

            # Obtener coordenadas del punto inicial y final de la conexión
            start_point = (int(keypoints[start_keypoint][0]), int(keypoints[start_keypoint][1]))
            end_point = (int(keypoints[end_keypoint][0]), int(keypoints[end_keypoint][1]))

            # Dibujar línea entre los puntos
            cv2.line(frame, start_point, end_point, (255, 0, 0), 2)

            # Dibujar puntos de inicio y fin de la conexión
            cv2.circle(frame, start_point, 5, (0, 255, 0), -1)
            cv2.circle(frame, end_point, 5, (0, 255, 0), -1)

        if len(keypoints) == 17:


            # Extract relevant angles and distances
            left_shoulder = keypoints[5]
            right_shoulder = keypoints[6]
            left_elbow = keypoints[7]
            right_elbow = keypoints[8]
            left_wrist = keypoints[9]
            right_wrist = keypoints[10]
            left_hip = keypoints[11]
            right_hip = keypoints[12]
            left_knee = keypoints[13]
            right_knee = keypoints[14]
            left_ankle = keypoints[15]
            right_ankle = keypoints[16]

            # Example angles
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            
            # Example distances
            wrist_distance = calculate_distance(left_wrist, right_wrist)
            ankle_distance = calculate_distance(left_ankle, right_ankle)
            shoulder_distance = calculate_distance(left_shoulder, right_shoulder)
            hip_distance = calculate_distance(left_hip, right_hip)

            features = np.array(keypoints).flatten().tolist()

            X = scaler_X.transform(np.array(features).reshape(1, -1))
            X = smooth_data(X)

            predictions = model.predict(X)

            # Separar las predicciones
            predicciones_continuas = predictions[0][0]
            prediccion_clase = encoder.inverse_transform(np.argmax(predictions[1], axis=1))

            features_video = [left_elbow_angle, right_elbow_angle, left_shoulder_angle, right_shoulder_angle, left_knee_angle, right_knee_angle, wrist_distance, ankle_distance, shoulder_distance, hip_distance]


            # Comparar características del video con las predichas
            porcentaje_diferencia = 0.2 
            caracteristicas = ["Angulo del codo izquierdo", "Angulo del codo derecho", "Angulo del hombro izquierdo", "Angulo del hombro derecho", "Angulo de la rodilla izquierda", "Angulo de la rodilla derecha", "Distancia entre munecas", "Distancia entre tobillos", "Distancia entre hombros", "Distancia entre caderas"]


            array1 = np.array(predicciones_continuas)
            array2 = np.array(smooth_data(features_video))

            relacion = np.abs(1 - (array1 / array2))

            errores = [caracteristicas[i] for i in range(len(relacion)) if porcentaje_diferencia< relacion[i] < (1-porcentaje_diferencia)]


            clase_predicha = prediccion_clase[0]
            porcentaje_prediccion = np.max(predictions[1]) * 100

            # Mostrar la predicción de clase
            texto_clasificacion = f'Clasificacion: {clase_predicha} - Probabilidad: {porcentaje_prediccion:.2f}%'
            cv2.putText(frame, texto_clasificacion, (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            if len(errores) > 0:
                for i,error in enumerate(errores):
                    desplazamiento_vertical = 240 + (i * 30)  # Incrementa la posición vertical para cada error adicional
                    cv2.putText(frame, f'Error en : {error}', (50, desplazamiento_vertical), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)


        cv2.imshow('Frame', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

        # Listen for 'q' key to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# Cargar el modelo y las clases
ruta_actual = os.getcwd()
carpeta_Yolo= os.path.dirname(ruta_actual)
ruta_raiz = os.path.dirname(carpeta_Yolo)

ruta_dataset = os.path.join(ruta_raiz, 'Data Set') 
ruta_video = os.path.join(ruta_dataset, 'test_video_4.mp4')
ruta_npy = os.path.join(ruta_actual, 'datos', 'classes.npy')
ruta_scaler = os.path.join(ruta_actual,'datos','scaler_X.pkl')
ruta_modelo = os.path.join(ruta_actual, 'model', 'exercise_model.h5')

modelYolo = YOLO("yolov8n-pose.pt")

# Cargar el modelo y scaler
model = load_model(ruta_modelo)
scaler_X = joblib.load(ruta_scaler)

classes = np.load(ruta_npy, allow_pickle=True)
encoder = LabelEncoder()
encoder.classes_ = classes

# Realizar la predicción
extract_pose_features(ruta_video,model,scaler_X,encoder,modelYolo)



0: 384x640 1 person, 135.7ms
Speed: 4.0ms preprocess, 135.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step

0: 384x640 1 person, 98.9ms
Speed: 3.0ms preprocess, 98.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

0: 384x640 1 person, 95.6ms
Speed: 3.0ms preprocess, 95.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

0: 384x640 1 person, 97.9ms
Speed: 3.0ms preprocess, 97.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

0: 384x640 1 person, 101.0ms
Speed: 3.0ms preprocess, 101.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

0: 384x640 1 person, 90.4ms
Speed: 2.0ms preprocess, 90.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

0: 384x640